In [102]:
#!pip install pingouin

In [103]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import calendar
#import pingouin as pg
from datetime import datetime
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [111]:
from google.colab import drive
drive.mount('/content/gdrive')

# directory
%cd '/content/gdrive/My Drive/Oxford/ML_for_Social_Good'

# import
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

train["Set"] = "train"
test["Set"] = "test"

df = pd.concat([train, test])
print(df.shape)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Oxford/ML_for_Social_Good
(5160, 45)


In [112]:
datetime_cols = ["CropTillageDate","RcNursEstDate","SeedingSowingTransplanting","Harv_date","Threshing_date"]

for col in datetime_cols:
  df[col] = pd.to_datetime(df[col])#.dt.date

In [113]:
# One row has Jamui as district but Gurua as Block, which is a Gaya block -- correcting its District
df.loc[(df["District"]=="Jamui") & (df["Block"]=="Gurua")].index
df.loc[2177,"District"] = "Gaya"

In [114]:
# OUTLIERS -- first pass

# SeedlingsPerPit has two extreme outliers (800 seedlings & 442 seedlings) --> replacing with the next max value for now (=22)
df["SeedlingsPerPit"] = df["SeedlingsPerPit"].replace(800,22).replace(442,22)

# TransplantingIrrigationHours
df.loc[df["TransplantingIrrigationHours"]>90]["TransplantingIrrigationHours"] = 90

# TransIrriCost have several extreme outliers (e.g. 6000 rupees for an average sized land)
#df["TransIrriCost"] = df["TransIrriCost"].replace(6000,3000)
df.loc[df["TransIrriCost"]>2500]["TransIrriCost"] = 2500

# Ganaura
df.loc[df["Ganaura"]>140]["Ganaura"] = 140

# 1appDaysUrea -- replacing extreme outlier with the next max value
df["1appDaysUrea"] = df["1appDaysUrea"].replace(332,75)

In [115]:
# PER-ACRE COLUMNS

list_cols = ["TransIrriCost","Ganaura","CropOrgFYM","BasalDAP","BasalUrea","1tdUrea","2tdUrea","Harv_hand_rent","Yield"]

for col in list_cols:
  label = str(col) + "_per_Acre"
  df[label] = df[col] / df["Acre"]

In [116]:
# Re-ordering columns and dropping the non-standardized ones
df = df[['ID', 'Set', 'District', 'Block', 'LandPreparationMethod', 'CropTillageDate', 'CropTillageDepth','CropEstMethod', 'RcNursEstDate',
         'SeedingSowingTransplanting','SeedlingsPerPit', 'NursDetFactor', 'TransDetFactor','TransplantingIrrigationHours', 'TransplantingIrrigationSource',
         'TransplantingIrrigationPowerSource', "TransIrriCost", 'TransIrriCost_per_Acre', 'StandingWater','OrgFertilizers', 'Ganaura', 'Ganaura_per_Acre', 'CropOrgFYM',
         'CropOrgFYM_per_Acre', 'PCropSolidOrgFertAppMethod', 'NoFertilizerAppln', 'CropbasalFerts', "BasalDAP", 'BasalDAP_per_Acre', "BasalUrea", 'BasalUrea_per_Acre',
         'MineralFertAppMethod', 'FirstTopDressFert', "1tdUrea",'1tdUrea_per_Acre', '1appDaysUrea', "2tdUrea", '2tdUrea_per_Acre', '2appDaysUrea', 'MineralFertAppMethod.1',
         'Harv_method', 'Harv_date', "Harv_hand_rent", 'Harv_hand_rent_per_Acre', 'Threshing_date', 'Threshing_method','Residue_length', 'Residue_perc', 'Stubble_use',
         'Acre', 'Yield','Yield_per_Acre'
         ]]

In [117]:
# PARSING MESSY CATEGORICAL VARIABLES

# 1. LandPreparationMethod
#methods = ["TractorPlough","FourWheelTracRotavator","WetTillagePuddling","BullockPlough","Other"]

df["LandPrepMethod_TractorPlough"] = df["LandPreparationMethod"].str.contains("TractorPlough")
df["LandPrepMethod_FourWheelTracRotavator"] = df["LandPreparationMethod"].str.contains("FourWheelTracRotavator")
df["LandPrepMethod_WetTillagePuddling"] = df["LandPreparationMethod"].str.contains("WetTillagePuddling")
df["LandPrepMethod_BullockPlough"] = df["LandPreparationMethod"].str.contains("BullockPlough")
df["LandPrepMethod_Other"] = df["LandPreparationMethod"].str.contains("Other")


# 2. NursDetFactor
#reasons = ["CalendarDate","PreMonsoonShowers","IrrigWaterAvailability","LabourAvailability","SeedAvailability"]

df["NursDetFactor_CalendarDate"] = df["NursDetFactor"].str.contains("CalendarDate")
df["NursDetFactor_PreMonsoonShowers"] = df["NursDetFactor"].str.contains("PreMonsoonShowers")
df["NursDetFactor_IrrigWaterAvailability"] = df["NursDetFactor"].str.contains("IrrigWaterAvailability")
df["NursDetFactor_LabourAvailability"] = df["NursDetFactor"].str.contains("LabourAvailability" or "LaborAvailability")
df["NursDetFactor_SeedAvailability"] = df["NursDetFactor"].str.contains("SeedAvailability")


# 2. TransDetFactor
#reasons = ["LaborAvailability","CalendarDate","RainArrival","IrrigWaterAvailability","SeedlingAge"] # I think that's all of them

df["TransDetFactor_LabourAvailability"] = df["TransDetFactor"].str.contains("LabourAvailability" or "LaborAvailability")
df["TransDetFactor_CalendarDate"] = df["TransDetFactor"].str.contains("CalendarDate")
df["TransDetFactor_RainArrival"] = df["TransDetFactor"].str.contains("RainArrival")
df["TransDetFactor_IrrigWaterAvailability"] = df["TransDetFactor"].str.contains("IrrigWaterAvailability")
df["TransDetFactor_SeedlingAge"] = df["TransDetFactor"].str.contains("SeedlingAge")


# 3. CropbasalFerts
df["CropbasalFerts"] = df["CropbasalFerts"].fillna("None")
fertilizer_types = ["Urea","DAP","Other","NPK","MoP","NPKS","SSP","None"]

for fertilizer in fertilizer_types:
  label = "CropbasalFerts_" + fertilizer
  df[label] = df["CropbasalFerts"].str.contains(fertilizer)


# 4. FirstTopDressFert
df["FirstTopDressFert"] = df["FirstTopDressFert"].fillna("None")
fertilizer_types2 = ["Urea","DAP","NPK","NPKS","SSP","Other"]

for fertilizer in fertilizer_types2:
  label = "FirstTopDressFert_" + fertilizer
  df[label] = df["FirstTopDressFert"].str.contains(fertilizer)


# 5. OrgFertilizers
df["OrgFertilizers"] = df["OrgFertilizers"].fillna("None")
orgfertilizers = ["Ganaura","FYM","VermiCompost","Pranamrit","Ghanajeevamrit","Jeevamrit","PoultryManure"]
for fertilizer in orgfertilizers:
  label = "OrgFertilizers_" + fertilizer
  df[label] = df["OrgFertilizers"].str.contains(fertilizer)


# 6. Replacing all NaNs with False
cols = ['LandPrepMethod_TractorPlough','LandPrepMethod_FourWheelTracRotavator','LandPrepMethod_WetTillagePuddling', 'LandPrepMethod_BullockPlough','LandPrepMethod_Other',
        'NursDetFactor_CalendarDate','NursDetFactor_PreMonsoonShowers','NursDetFactor_IrrigWaterAvailability','NursDetFactor_LabourAvailability', 'NursDetFactor_SeedAvailability',
        'TransDetFactor_LabourAvailability', 'TransDetFactor_CalendarDate','TransDetFactor_RainArrival', 'TransDetFactor_IrrigWaterAvailability','TransDetFactor_SeedlingAge',
        'CropbasalFerts_Urea','CropbasalFerts_DAP', 'CropbasalFerts_Other', 'CropbasalFerts_NPK','CropbasalFerts_MoP', 'CropbasalFerts_NPKS', 'CropbasalFerts_SSP',
        'CropbasalFerts_None', 'FirstTopDressFert_Urea','FirstTopDressFert_DAP', 'FirstTopDressFert_NPK','FirstTopDressFert_NPKS', 'FirstTopDressFert_SSP','FirstTopDressFert_Other',
        'OrgFertilizers_Ganaura','OrgFertilizers_FYM', 'OrgFertilizers_VermiCompost','OrgFertilizers_Pranamrit', 'OrgFertilizers_Ghanajeevamrit','OrgFertilizers_Jeevamrit',
        'OrgFertilizers_PoultryManure']

for col in cols:
  df[col] = df[col].fillna(False)

In [118]:
# checking there are no missing values left among parsed categoricals
test = df[['LandPrepMethod_TractorPlough','LandPrepMethod_FourWheelTracRotavator','LandPrepMethod_WetTillagePuddling', 'LandPrepMethod_BullockPlough','LandPrepMethod_Other',
        'NursDetFactor_CalendarDate','NursDetFactor_PreMonsoonShowers','NursDetFactor_IrrigWaterAvailability','NursDetFactor_LabourAvailability', 'NursDetFactor_SeedAvailability',
        'TransDetFactor_LabourAvailability', 'TransDetFactor_CalendarDate','TransDetFactor_RainArrival', 'TransDetFactor_IrrigWaterAvailability','TransDetFactor_SeedlingAge',
        'CropbasalFerts_Urea','CropbasalFerts_DAP', 'CropbasalFerts_Other', 'CropbasalFerts_NPK','CropbasalFerts_MoP', 'CropbasalFerts_NPKS', 'CropbasalFerts_SSP',
        'CropbasalFerts_None', 'FirstTopDressFert_Urea','FirstTopDressFert_DAP', 'FirstTopDressFert_NPK','FirstTopDressFert_NPKS', 'FirstTopDressFert_SSP','FirstTopDressFert_Other',
        'OrgFertilizers_Ganaura','OrgFertilizers_FYM', 'OrgFertilizers_VermiCompost','OrgFertilizers_Pranamrit', 'OrgFertilizers_Ghanajeevamrit','OrgFertilizers_Jeevamrit',
        'OrgFertilizers_PoultryManure']]

test.isnull().sum()

LandPrepMethod_TractorPlough             0
LandPrepMethod_FourWheelTracRotavator    0
LandPrepMethod_WetTillagePuddling        0
LandPrepMethod_BullockPlough             0
LandPrepMethod_Other                     0
NursDetFactor_CalendarDate               0
NursDetFactor_PreMonsoonShowers          0
NursDetFactor_IrrigWaterAvailability     0
NursDetFactor_LabourAvailability         0
NursDetFactor_SeedAvailability           0
TransDetFactor_LabourAvailability        0
TransDetFactor_CalendarDate              0
TransDetFactor_RainArrival               0
TransDetFactor_IrrigWaterAvailability    0
TransDetFactor_SeedlingAge               0
CropbasalFerts_Urea                      0
CropbasalFerts_DAP                       0
CropbasalFerts_Other                     0
CropbasalFerts_NPK                       0
CropbasalFerts_MoP                       0
CropbasalFerts_NPKS                      0
CropbasalFerts_SSP                       0
CropbasalFerts_None                      0
FirstTopDre

In [121]:
print(df.OrgFertilizers.isnull().sum())
print(df.FirstTopDressFert.isnull().sum())
print(df.TransDetFactor.isnull().sum())

0
0
392


In [120]:
# MISSING VALUES

df["2tdUrea"] = df["2tdUrea"].fillna(0)
df["1tdUrea"] = df["1tdUrea"].fillna(0)
df["Harv_hand_rent"] = df["Harv_hand_rent"].fillna(0)
df["Ganaura"] = df["Ganaura"].fillna(0)
df["CropOrgFYM"] = df["CropOrgFYM"].fillna(0)
df["BasalDAP"] = df["BasalDAP"].fillna(0)
df["BasalUrea"] = df["BasalUrea"].fillna(0)

# Creating a new variable counting the number of missing values for each row
df["Nb_of_NaN"] = df.drop(columns=["Yield","Yield_per_Acre"]).isnull().sum(axis=1)
# in case that's useful to define a threshold and drop rows that are too incomplete

In [ ]:
# Dropping messy cols
df = df.drop(columns=["LandPreparationMethod","NursDetFactor","TransDetFactor","OrgFertilizers","CropbasalFerts","FirstTopDressFert"])

In [125]:
# EXPORTING

# V1: full cleaned df
df.to_csv('cleaned_fulldf.csv',index=False)

# V2: per Acre df (dropping the raw variables)
#df.copy().drop(columns=["TransIrriCost","Ganaura","CropOrgFYM","BasalDAP","BasalUrea","1tdUrea","2tdUrea","Harv_hand_rent","Yield"]).to_csv('peracre_df.csv',index=False)

# V2: per Acre df (dropping the raw variables)
#df.copy().drop(columns=["TransIrriCost_per_Acre","Ganaura_per_Acre","CropOrgFYM_per_Acre","BasalDAP_per_Acre","BasalUrea_per_Acre","1tdUrea_per_Acre","2tdUrea_per_Acre",
#                        "Harv_hand_rent_per_Acre","Yield_per_Acre"]).to_csv('rawyield_df.csv',index=False)

In [127]:
df.shape

(5160, 83)

#### Missing values - Mann-Whitney U tests

-- won't run anymore, bc removed missing values in above cells

In [62]:
# Mann-Whitney Utests: NaNs vs non-NaNs on Yield_per_Acre, for all variables with missing values

# Selecting only train set
train = df.loc[df["Set"]=="train"]

# List of columns with missing values
var_list = ["RcNursEstDate","SeedlingsPerPit","NursDetFactor","TransDetFactor","TransplantingIrrigationHours","TransplantingIrrigationSource","TransplantingIrrigationPowerSource",
            "TransIrriCost","StandingWater","OrgFertilizers","Ganaura","CropOrgFYM","PCropSolidOrgFertAppMethod","BasalDAP","BasalUrea","FirstTopDressFert","1tdUrea",
            "2tdUrea","MineralFertAppMethod.1","Harv_hand_rent"
            ]

# Run Mann-Whitney U tests for all variables with missing values
for variable in var_list:
  na = train[train[variable].isnull()==True]
  notna = train[train[variable].isnull()==False]
  print("\n",variable)
  print("NA Group n=",na.shape[0], "\nNot NA Group n=",notna.shape[0])
  print("NA Group median=",na.Yield_per_Acre.median().round(2), "\nNot NA Group median=",notna.Yield_per_Acre.median().round(2))
  print(pg.mwu(na["Yield_per_Acre"], notna["Yield_per_Acre"], alternative='two-sided'))


 RcNursEstDate
NA Group n= 83 
Not NA Group n= 3787
NA Group median= 2560.0 
Not NA Group median= 1870.0
        U-val alternative         p-val       RBC      CLES
MWU  257721.0   two-sided  1.660096e-23 -0.639859  0.819929

 SeedlingsPerPit
NA Group n= 289 
Not NA Group n= 3581
NA Group median= 1805.71 
Not NA Group median= 1898.67
        U-val alternative     p-val       RBC      CLES
MWU  471243.0   two-sided  0.011394  0.089305  0.455347

 NursDetFactor
NA Group n= 289 
Not NA Group n= 3581
NA Group median= 1805.71 
Not NA Group median= 1898.67
        U-val alternative     p-val       RBC      CLES
MWU  471243.0   two-sided  0.011394  0.089305  0.455347

 TransDetFactor
NA Group n= 289 
Not NA Group n= 3581
NA Group median= 1805.71 
Not NA Group median= 1898.67
        U-val alternative     p-val       RBC      CLES
MWU  471243.0   two-sided  0.011394  0.089305  0.455347

 TransplantingIrrigationHours
NA Group n= 193 
Not NA Group n= 3677
NA Group median= 1920.0 
Not NA Group m